In [1]:
from collections import OrderedDict
import argparse
import re
from scipy.stats import gmean

pattern_map = {
    'runtime': 'simTicks\s+\d+',
    'network_lat': 'system.ruby.network.average_packet_network_latency\s+\d+',
    'inst': 'simInsts\s+\d+',
    'miss_stale': 'system.ruby.l2_cntrl\d+.L2cache.m_fwd_issued\s+\d+',
    'miss_total': 'system.ruby.l2_cntrl\d+.L2cache.m_demand_misses\s+\d+',
    'hit_total': 'system.ruby.l2_cntrl\d+.L2cache.m_demand_hits\s+\d+',
    'access_total': 'system.ruby.l2_cntrl\d+.L2cache.m_demand_accesses\s+\d+',
}

prune_map = {
    'runtime': lambda line: float(line.replace(' ','').split('simTicks')[-1].split('#')[0]),
    'network_lat': lambda line: float(line.replace(' ','').split('system.ruby.network.average_packet_network_latency')[-1].split('(')[0]),
    'inst': lambda line: float(line.replace(' ','').split('simInsts')[-1].split('#')[0]),
    'miss_stale': lambda line: int(line.replace(' ','').split('m_fwd_issued')[-1].split('#')[0]),
    'miss_total': lambda line: int(line.replace(' ','').split('m_demand_misses')[-1].split('#')[0]),
    'hit_total': lambda line: int(line.replace(' ','').split('m_demand_hits')[-1].split('#')[0]),
    'access_total': lambda line: int(line.replace(' ','').split('m_demand_accesses')[-1].split('#')[0]),
}

In [2]:
import plotly.io as pio
pio.renderers.default = "browser"

In [3]:

time_str = '20220608140651'
bench = ['blackscholes', 'bodytrack', 'canneal', 'dedup', \
         'ferret', 'fluidanimate', \
#          'freqmine', \
#          'streamcluster', 'swaptions', \
        'raytrace', \
#          'vips',\
        ]
sim_str = 'simsmall'
config = [4]
outdir = [f'{time_str}/gem5/my_STATS/base/{b}_{sim_str}_64_{c}/stats.txt' for b in bench for c in config ]

In [4]:
def get_list(file, metric):
    pattern = pattern_map[metric]

    list_ = []
    with open(file, 'r') as searchfile:
        for line in searchfile:
            if re.match(pattern, line):
                prune_func = prune_map[metric]
                res = prune_func(line)
                list_.append(res)
    # print(list_)
    return list_

In [5]:
o_arr = [o.split('/')[4].split('_64')[0] for o in outdir]
print(f'benchmarks={o_arr}')

benchmarks=['blackscholes_simsmall', 'bodytrack_simsmall', 'canneal_simsmall', 'dedup_simsmall', 'ferret_simsmall', 'fluidanimate_simsmall', 'raytrace_simsmall']


In [6]:
metric = 'miss_stale'
miss_stale_l = [get_list(o, metric) for o in outdir if '_4' in o]
# print(f'miss stale list ={miss_stale_l}\n')

metric = 'miss_total'
miss_total_l = [get_list(o, metric) for o in outdir if '_4' in o]
# print(f'miss total list ={miss_total_l}\n')

miss_np_l = [[t-s for t,s in zip(t_l, s_l)] for t_l,s_l in zip(miss_total_l, miss_stale_l)]
# print(f'miss not present list ={miss_np_l}\n')

metric = 'hit_total'
hit_total_l = [get_list(o, metric) for o in outdir if '_4' in o]
# print(f'hit total list ={hit_total_l}\n')

metric = 'access_total'
access_total_l = [get_list(o, metric) for o in outdir if '_4' in o]
# print(f'access total list ={access_total_l}\n')

In [7]:
# miss_s_percent = [[float(s/a*100) for a,s in zip(a_l, s_l)] for a_l,s_l in zip(access_total_l, miss_stale_l)]
# miss_s_gm = [gmean(l) for l in miss_s_percent]
# print(f'miss stale percent ={miss_s_percent}, gm={miss_s_gm}\n')

# miss_np_percent = [[float(np/a*100) for a,np in zip(a_l, np_l)] for a_l,np_l in zip(access_total_l, miss_np_l)]
# miss_np_gm = [gmean(l) for l in miss_np_percent]
# print(f'miss not present percent ={miss_np_percent}, gm={miss_np_gm}\n')

# hit_percent = [[float(h/a*100) for a,h in zip(a_l, h_l)] for a_l,h_l in zip(access_total_l, hit_total_l)]
# hit_gm = [gmean(l) for l in hit_percent]
# print(f'hit percent ={hit_percent}, gm={hit_gm}\n')

In [8]:
miss_s = [sum(l) for l in miss_stale_l]
# print(f'total miss stale across cores={miss_s}')

miss_np = [sum(l) for l in miss_np_l]
# print(f'total miss np across cores={miss_np}')

hit = [sum(l) for l in hit_total_l]
# print(f'total hit across cores={hit}')

access = [sum(l) for l in access_total_l]
# print(f'total access across cores={access}')

In [9]:
miss_s_percent = [float(part/total)*100 for part, total in zip(miss_s, access)]
miss_np_percent = [float(part/total)*100 for part, total in zip(miss_np, access)]
hit_percent = [float(part/total)*100 for part, total in zip(hit, access)]
print(f'miss_s={miss_s_percent}\nmiss_np={miss_np_percent}\nhit={hit_percent}')

miss_s_gm = gmean(miss_s_percent)
miss_np_gm = gmean(miss_np_percent)
hit_gm = gmean(hit_percent)


miss_s=[13.7411417843355, 2.0739250741607984, 9.3108010305474, 5.743068060420702, 3.094852495119144, 21.448094879475335, 4.145657590585692]
miss_np=[19.99609974086414, 2.8251796471719692, 15.326240594485615, 10.644029528458898, 14.88326672947582, 8.235122541992864, 37.57293386569001]
hit=[66.26275847480036, 95.10089527866722, 75.36295837496698, 83.6129024111204, 82.02188077540504, 70.3167825785318, 58.2814085437243]


In [10]:
print(f'\n=== summary ===')
print(f'miss stale gm={miss_s_gm}')
print(f'miss np gm={miss_np_gm}')
print(f'hit gm={hit_gm}')


=== summary ===
miss stale gm=6.356589154308499
miss np gm=12.293692864197466
hit gm=74.99938975461082


In [11]:
o_arr.append('geomean')
miss_s_percent.append(miss_s_gm)
miss_np_percent.append(miss_np_gm)
hit_percent.append(hit_gm)

In [12]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='miss_state', x=o_arr, y=miss_s_percent),
    go.Bar(name='miss_notPresent', x=o_arr, y=miss_np_percent),
    go.Bar(name='hit', x=o_arr, y=hit_percent),
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.update_layout(title_text='LLC Access; misses due to stale triggers fwd message generation in MESI')
fig.show()